In [16]:
import yfinance as yf
import pandas as pd
import ta
import datetime
from ta import momentum, trend
import math

class RSIAnalysis:
    def __init__(self, symbol, start_date, end_date):
        self.symbol = symbol
        self.start_date = start_date
        self.end_date = end_date
        self.interval_data = None

    def download_data(self):
        # Download interval data (15 minutes)
        self.interval_data = yf.download(self.symbol, start=self.start_date, end=self.end_date, interval='5m')
        self.interval_data.drop(columns=['Adj Close', 'Volume'], inplace=True)
        self.interval_data = self.interval_data.astype(int)

        # Download daily data
        self.daily_data = yf.download(self.symbol, start=self.start_date, end=datetime.date.today(), interval='1d')
        self.daily_data = self.daily_data.astype(int)

    def add_previous_day_data(self, row):
        date = row.name.date()
        previous_date = date - pd.DateOffset(days=1)

        if previous_date in self.daily_data.index:
            previous_day_data = self.daily_data.loc[previous_date]
            row['Previous_Day_Open'] = previous_day_data['Open']
            row['Previous_Day_High'] = previous_day_data['High']
            row['Previous_Day_Low'] = previous_day_data['Low']
            row['Previous_Day_Close'] = previous_day_data['Close']

        return row

    def calculate_daily_high_low(self):
        self.daily_data['Daily_High'] = self.daily_data['High'].shift()
        self.daily_data['Daily_Low'] = self.daily_data['Low'].shift()

        for date, daily_row in self.daily_data.iterrows():
            mask = self.interval_data.index.date == date.date()
            self.interval_data.loc[mask, 'Daily_High'] = daily_row['Daily_High']
            self.interval_data.loc[mask, 'Daily_Low'] = daily_row['Daily_Low']

    def calculate_rsi_strategy(self, rsi_window=4):
        self.interval_data['RSI'] = ta.momentum.RSIIndicator(self.interval_data['Close'], window=rsi_window).rsi()

        def rsi_strategy(row):
            row_dict = row[['Previous_Day_High', 'Previous_Day_Low', 'Previous_Day_Close', 'Daily_High', 'Daily_Low']].to_dict()
            close = row['Close']
            min_key = min(row_dict, key=lambda key: abs(row_dict[key] - close))
            min_value = row_dict[min_key]
            min_difference = abs(min_value - close)

            if (row['RSI'] <= 30) and min_difference <= 50:
                return "Bullish"
            elif (row['RSI'] >= 70) and min_difference <= 50:
                return "Bearish"
            else:
                return None

        self.interval_data['RSI Status'] = self.interval_data.apply(rsi_strategy, axis=1)
        def Find_Up(df):
            data = df[0] < df[1]
            if data == True:
                return "Green"
            else:
                return "Red"


        self.interval_data['Canndel'] = self.interval_data[['Open', 'Close']].apply(Find_Up, axis=1)
        self.interval_data['SMA20'] = self.interval_data['Close'].rolling(window=20).mean()
        self.interval_data['SMA40'] = self.interval_data['Close'].rolling(window=40).mean()
        self.interval_data['RSI'] = momentum.RSIIndicator(self.interval_data['Close'], window=5).rsi()
        self.interval_data['Trend'] = 'Downtrend' 
        self.interval_data.loc[self.interval_data['SMA20'] > self.interval_data['SMA40'], 'Trend'] = 'Uptrend'

        self.interval_data['MACD_Blue'] = trend.MACD(self.interval_data['Close']).macd()
        self.interval_data['MACD_Red'] = trend.MACD(self.interval_data['Close']).macd_signal()
        self.interval_data['MACD_Value'] = self.interval_data['MACD_Blue'] - self.interval_data['MACD_Red']
        
        self.interval_data['RollingStd'] = self.interval_data['Close'].rolling(window=20).std()
 
        self.interval_data['B_UpperBand'] = self.interval_data['SMA20'] + (self.interval_data['RollingStd'] * 2)
        self.interval_data['B_LowerBand'] = self.interval_data['SMA20'] - (self.interval_data['RollingStd'] * 2)
        
        self.interval_data['Bolinger Status'] = 'Neutral'  # Initialize with 'Neutral', you can change this to any default value

        self.interval_data.loc[self.interval_data['High'] > self.interval_data['B_UpperBand'], 'Bolinger Status'] = 'Bearish'
        self.interval_data.loc[self.interval_data['Low'] < self.interval_data['B_LowerBand'], 'Bolinger Status'] = 'Bullish'
            
        self.interval_data = self.interval_data .drop(columns=["SMA20", "SMA40"], axis=1)
        self.interval_data['ADX'] = ta.trend.ADXIndicator(self.interval_data['High'], self.interval_data['Low'], self.interval_data['Close'], window=14).adx()
        self.interval_data = self.interval_data.sort_values(by='Datetime', ascending=False)

    def run_analysis(self):
        self.download_data()
        self.interval_data = self.interval_data.apply(self.add_previous_day_data, axis=1)
        self.calculate_daily_high_low()
        self.calculate_rsi_strategy()

if __name__ == "__main__":
    symbol = '^NSEI'
    start_date = pd.Timestamp.now() - pd.DateOffset(days=10)
    end_date = pd.Timestamp.now()
    analysis = RSIAnalysis(symbol, start_date, end_date)
    analysis.run_analysis()
analysis.interval_data.head(5)


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


C:\Python\Lib\site-packages\ta\trend.py:780: RuntimeWarning: invalid value encountered in scalar divide
  dip[idx] = 100 * (self._dip[idx] / value)
C:\Python\Lib\site-packages\ta\trend.py:785: RuntimeWarning: invalid value encountered in scalar divide
  din[idx] = 100 * (self._din[idx] / value)


,Close,High,Low,Open,Previous_Day_Close,Previous_Day_High,Previous_Day_Low,Previous_Day_Open,Daily_High,Daily_Low,...,Canndel,Trend,MACD_Blue,MACD_Red,MACD_Value,RollingStd,B_UpperBand,B_LowerBand,Bolinger Status,ADX
Datetime,,,,,,,,,,,,,,,,,,,,,
2023-09-29 15:25:00+05:30,19638.0,19640.0,19623.0,19626.0,19523.0,19766.0,19492.0,19761.0,19766.0,19492.0,...,Green,Uptrend,-11.973249,-2.182446,-9.790803,35.991629,19755.633258,19611.666742,Neutral,36.265950
2023-09-29 15:20:00+05:30,19626.0,19641.0,19625.0,19634.0,19523.0,19766.0,19492.0,19761.0,19766.0,19492.0,...,Red,Uptrend,-11.031691,0.265254,-11.296945,34.353082,19754.856163,19617.443837,Neutral,35.767152
2023-09-29 15:15:00+05:30,19634.0,19635.0,19618.0,19624.0,19523.0,19766.0,19492.0,19761.0,19766.0,19492.0,...,Green,Uptrend,-8.447642,3.089491,-11.537132,31.420912,19751.541823,19625.858177,Bullish,35.314534
2023-09-29 15:10:00+05:30,19623.0,19636.0,19623.0,19632.0,19523.0,19766.0,19492.0,19761.0,19766.0,19492.0,...,Red,Uptrend,-5.841126,5.973774,-11.814899,28.699670,19748.649340,19633.850660,Bullish,34.211412
2023-09-29 15:05:00+05:30,19632.0,19657.0,19631.0,19644.0,19523.0,19766.0,19492.0,19761.0,19766.0,19492.0,...,Red,Uptrend,-1.309301,8.927498,-10.236800,24.012935,19742.125870,19646.074130,Bullish,33.200505
